<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração


In [ ]:
import requests
import re
import pandas as pd

## Extrair cógidos FIPE

Feito manualmente, todos os códigos FIPE estão presentes na pagina:
https://www.tabelafipebrasil.com/fipe/carros
bastou fazer um regex no formato [\d]+-[\d]


arquivo gerado ficou diisponível em "https://github.com/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/FIPE-CODES.txt"

In [ ]:
def get_fipe_codes():
  codes_string = requests.get("https://raw.githubusercontent.com/JoaoVitorSantiagoNogueira/case-brick-seguros/main/FIPE-CODES.txt").text
  codes_list = re.split("\r\n",codes_string)
  return codes_list

## Pegar Informações de Veiculo

### Métodos de extração

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor

def get_url(url):
    return requests.get(url)

def get_url_batch(list_of_urls):
  with ThreadPoolExecutor(max_workers=50) as pool:
      return list(pool.map(get_url,list_of_urls))

In [ ]:
def FIPE_url_list_generator(codes, query):

  base_url = "https://brasilapi.com.br/api/fipe/preco/v1/"
  Q = []*4

  all_urls = []
  for c, q in zip(codes, query):
    if q == "":
      q=0
    for i in range(0,4):
      all_urls.append(base_url+c+"?tabela_referencia="+str(q+i))

  return all_urls

## métodos de seleção

In [ ]:
def batch_info_selector(num_elements, options, date_range):

  import random

  # Getting Random N lists of size K
  selected_elements = []
  selected_dates = []

  selected_elements = random.sample(options, num_elements)
  selected_dates    = [random.randrange(date_range) for i in range(num_elements)]

  return FIPE_url_list_generator(selected_elements,selected_dates)

### Criação do Json base

In [ ]:
test_url_list = batch_info_selector(3000,                ##numero arbirtário para um "bom treinamento", chutando 3000 (5000 estoura ram)
                                    get_fipe_codes(),   ##pega a lista com todos os códigos
                                    168)                ##dados começam a partir de 2006~, até 2020, ~168 entradas
sample_json_list = [j.json() for j in get_url_batch(test_url_list)]

['https://brasilapi.com.br/api/fipe/preco/v1/024047-8?tabela_referencia=138',
 'https://brasilapi.com.br/api/fipe/preco/v1/024047-8?tabela_referencia=139',
 'https://brasilapi.com.br/api/fipe/preco/v1/024047-8?tabela_referencia=140',
 'https://brasilapi.com.br/api/fipe/preco/v1/024047-8?tabela_referencia=141',
 'https://brasilapi.com.br/api/fipe/preco/v1/015043-6?tabela_referencia=9',
 'https://brasilapi.com.br/api/fipe/preco/v1/015043-6?tabela_referencia=10',
 'https://brasilapi.com.br/api/fipe/preco/v1/015043-6?tabela_referencia=11',
 'https://brasilapi.com.br/api/fipe/preco/v1/015043-6?tabela_referencia=12',
 'https://brasilapi.com.br/api/fipe/preco/v1/021197-4?tabela_referencia=16',
 'https://brasilapi.com.br/api/fipe/preco/v1/021197-4?tabela_referencia=17',
 'https://brasilapi.com.br/api/fipe/preco/v1/021197-4?tabela_referencia=18',
 'https://brasilapi.com.br/api/fipe/preco/v1/021197-4?tabela_referencia=19',
 'https://brasilapi.com.br/api/fipe/preco/v1/021114-1?tabela_referencia=1

# Préprocessamento

## métodos de processamento do JSON

In [ ]:
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
      flat_list.extend(row)
    return flat_list

In [ ]:
def group_jsons_by_field(jsons, fields):
  database_lines = []
  while jsons:
    ref = jsons[0]

    #primeira passada, encontra todos os elementos que batem
    match_list = []
    for i in range(0, len(jsons)):
      values_match =  True
      for field in fields:
        values_match = values_match and field in jsons[i] and jsons[i][field] == ref[field]

      if values_match:
        match_list.append(i)


    #remove as coincidências e junta tudo em uma tabela só
    matches = []
    for i in reversed(match_list):
      matches.append(jsons.pop(i))
    #junta elas numa entrada só
    database_lines.append(matches)
  return database_lines

In [ ]:
def merge_json_split_months_costs(jsons):
  new_keys = []
  new_values = []
  for json in jsons:
      new_keys.append(json.pop("mesReferencia"))
      new_values.append(json.pop("valor"))

  if len(new_keys)<4: ##ignora entradas sem a devida quantidade de valores. Poderiamos preencher, mas por simplicidade e para não enviezar os dados façamos isso.
    return

  #trocar para valor numérico no formato aaaa/mm
  months = ["janeiro de ","fevereiro de ","março de ","abril de ", "maio de ", "junho de ", "julho de ", "agosto de ", "setembro de ", "outubro de ", "novembro de ", "dezembro de "]
  month_numbers = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

  for key in range(0, len(new_keys)):
    for month_text, month_number in zip (months, month_numbers):
      new_keys[key] = new_keys[key].replace(month_text, month_number)

  new_keys = [i[2:-1]+i[:2] for i in new_keys]

  #ordena por data
  results = sorted(zip(new_keys, new_values))
  new_keys   = [i[0]for i in results]
  new_values = [i[1]for i in results]

  #atribui as datas como, mesDesejado , o valor target do Mes desejado, e o valor dos 3 mêses anteriores, se vazio, se faltar um dos 3 completa pela média dos demais
  jsons[0]["mesDesejado"] = new_keys  [3]
  jsons[0]["target"]      = int (re.sub("[^\d]", "", new_values[3]))//100
  jsons[0]["mes-1"]       = int (re.sub("[^\d]", "", new_values[2]))//100
  jsons[0]["mes-2"]       = int (re.sub("[^\d]", "", new_values[1]))//100
  jsons[0]["mes-3"]       = int (re.sub("[^\d]", "", new_values[0]))//100

  return jsons[0]

## Préprocessamento do JSON

In [ ]:
#processo de limpeza, planificar todos os itens, agrupar código fipe iguais, préprocessar os dados
sample_json_list = flatten_extend(sample_json_list )
sample_json_list = [item for item in sample_json_list if type(item) == dict]  #remover entradas que voltaram com erro
sample_json_list = group_jsons_by_field(sample_json_list,["codigoFipe", "modelo", "anoModelo" , "siglaCombustivel"])
sample_json_list = [merge_json_split_months_costs(item) for item in sample_json_list]
sample_json_list = [item for item in sample_json_list if type(item) == dict] #remover entradas que voltaram com erro

In [ ]:
sample_json_list

[{'marca': 'Jaguar',
  'modelo': 'X-Type SE 2.5 V6 194cv',
  'anoModelo': 32000,
  'combustivel': 'Gasolina',
  'codigoFipe': '016013-0',
  'tipoVeiculo': 1,
  'siglaCombustivel': 'G',
  'dataConsulta': 'terça-feira, 14 de maio de 2024 22:50',
  'mesDesejado': '200312',
  'target': 170554,
  'mes-1': 171372,
  'mes-2': 178026,
  'mes-3': 157356},
 {'marca': 'Jaguar',
  'modelo': 'X-Type SE 2.5 V6 194cv',
  'anoModelo': 2002,
  'combustivel': 'Gasolina',
  'codigoFipe': '016013-0',
  'tipoVeiculo': 1,
  'siglaCombustivel': 'G',
  'dataConsulta': 'terça-feira, 14 de maio de 2024 22:50',
  'mesDesejado': '200312',
  'target': 140493,
  'mes-1': 138554,
  'mes-2': 145381,
  'mes-3': 130418},
 {'marca': 'Buggy',
  'modelo': 'Buggy 1.6 2-Lug.',
  'anoModelo': 1997,
  'combustivel': 'Gasolina',
  'codigoFipe': '040001-7',
  'tipoVeiculo': 1,
  'siglaCombustivel': 'G',
  'dataConsulta': 'terça-feira, 14 de maio de 2024 22:50',
  'mesDesejado': '201212',
  'target': 6306,
  'mes-1': 6343,
  'me

## Transfere para o pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.json_normalize(sample_json_list)
df

,marca,modelo,anoModelo,combustivel,codigoFipe,tipoVeiculo,siglaCombustivel,dataConsulta,mesDesejado,target,mes-1,mes-2,mes-3
0,Jaguar,X-Type SE 2.5 V6 194cv,32000,Gasolina,016013-0,1,G,"terça-feira, 14 de maio de 2024 22:50",200312,170554,171372,178026,157356
1,Jaguar,X-Type SE 2.5 V6 194cv,2002,Gasolina,016013-0,1,G,"terça-feira, 14 de maio de 2024 22:50",200312,140493,138554,145381,130418
2,Buggy,Buggy 1.6 2-Lug.,1997,Gasolina,040001-7,1,G,"terça-feira, 14 de maio de 2024 22:50",201212,6306,6343,6372,6393
3,Buggy,Buggy 1.6 2-Lug.,1996,Gasolina,040001-7,1,G,"terça-feira, 14 de maio de 2024 22:50",201212,5552,5597,5639,5674
4,Buggy,Buggy 1.6 2-Lug.,1995,Gasolina,040001-7,1,G,"terça-feira, 14 de maio de 2024 22:50",201212,5184,5275,5352,5421
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,Ford,EcoSport XLS FREESTYLE 1.6 Flex 8V 5p,2007,Gasolina,003303-0,1,G,"terça-feira, 14 de maio de 2024 23:14",201402,27827,27869,28134,27720
5041,Ford,EcoSport XLS FREESTYLE 1.6 Flex 8V 5p,2006,Gasolina,003303-0,1,G,"terça-feira, 14 de maio de 2024 23:14",201402,26946,26987,27089,26715
5042,BMW,118iA 2.0 16V 136cv 3p,32000,Gasolina,009152-9,1,G,"terça-feira, 14 de maio de 2024 23:14",201301,90215,90835,91377,91802
5043,BMW,118iA 2.0 16V 136cv 3p,2012,Gasolina,009152-9,1,G,"terça-feira, 14 de maio de 2024 23:14",201301,80045,81510,83084,80020


## Préprocessamento pandas

#### Normalização

In [ ]:
dfMean = df[['target', "mes-1","mes-2","mes-3"]].stack().mean()
dfStd  = df[['target', "mes-1","mes-2","mes-3"]].stack().std()
dfMax  = df[['target', "mes-1","mes-2","mes-3"]].stack().max()
print(dfMean, dfStd, dfMax)


51807.21481665015 87225.80593145234 1700000


In [ ]:
#normalizedDf = (df[['target', "mes-1","mes-2","mes-3"]]-dfMean)/dfStd
#preferi não normalizar subtraindo a média já que valor é sempre positivo, preferi dividir pelo maior numero
normalizedDf = (df[['target', "mes-1","mes-2","mes-3"]])/dfMax
normalizedDf = pd.concat([df.drop(['target', "mes-1","mes-2","mes-3"], axis=1),normalizedDf], axis=1)
normalizedDf = normalizedDf.drop(['modelo', "combustivel","tipoVeiculo","dataConsulta"], axis=1)

NameError: name 'dfMax' is not defined

### Transformação de campos textuais em numéricos

In [ ]:
def get_lista_de_marcas():
  print("fetching brands")
  lista_de_marcas = requests.get("https://raw.githubusercontent.com/JoaoVitorSantiagoNogueira/case-brick-seguros/main/Top100CarBrands.txt").text
  lista_de_marcas = re.split("\r\n",lista_de_marcas)
  return lista_de_marcas

def rank_marca(x, lista_de_marcas):

  if type(x) == float:
    return x

  if lista_de_marcas == []:
    lista_de_marcas = get_lista_de_marcas()


  if x in lista_de_marcas:
    i = 100
    for item in lista_de_marcas:
      if x == item:
        return i/100
      i -= 1
  return 0

In [ ]:
normalizedDf["siglaCombustivel"] = normalizedDf["siglaCombustivel"].replace({'D': 1.0, 'G': 0.6, 'A': 0.3, 'Á':0.3})
normalizedDf["siglaCombustivel"] = normalizedDf["siglaCombustivel"].astype(float)
normalizedDf["marca"] = normalizedDf["marca"].apply(rank_marca, args=([get_lista_de_marcas()]))
normalizedDf["codigoFipe"] = normalizedDf["codigoFipe"].astype(str).str.replace('-', '').astype(float)
normalizedDf

fetching brands


,marca,anoModelo,codigoFipe,siglaCombustivel,mesDesejado,target,mes-1,mes-2,mes-3
0,0.56,32000,160130.0,0.6,200312,0.100326,0.100807,0.104721,0.092562
1,0.56,2002,160130.0,0.6,200312,0.082643,0.081502,0.085518,0.076716
2,0.00,1997,400017.0,0.6,201212,0.003709,0.003731,0.003748,0.003761
3,0.00,1996,400017.0,0.6,201212,0.003266,0.003292,0.003317,0.003338
4,0.00,1995,400017.0,0.6,201212,0.003049,0.003103,0.003148,0.003189
...,...,...,...,...,...,...,...,...,...
5040,0.92,2007,33030.0,0.6,201402,0.016369,0.016394,0.016549,0.016306
5041,0.92,2006,33030.0,0.6,201402,0.015851,0.015875,0.015935,0.015715
5042,0.96,32000,91529.0,0.6,201301,0.053068,0.053432,0.053751,0.054001
5043,0.96,2012,91529.0,0.6,201301,0.047085,0.047947,0.048873,0.047071


In [ ]:
normalizedDf

,marca,anoModelo,codigoFipe,siglaCombustivel,mesDesejado,target,mes-1,mes-2,mes-3
0,0.56,32000,160130.0,0.6,200312,0.100326,0.100807,0.104721,0.092562
1,0.56,2002,160130.0,0.6,200312,0.082643,0.081502,0.085518,0.076716
2,0.00,1997,400017.0,0.6,201212,0.003709,0.003731,0.003748,0.003761
3,0.00,1996,400017.0,0.6,201212,0.003266,0.003292,0.003317,0.003338
4,0.00,1995,400017.0,0.6,201212,0.003049,0.003103,0.003148,0.003189
...,...,...,...,...,...,...,...,...,...
5040,0.92,2007,33030.0,0.6,201402,0.016369,0.016394,0.016549,0.016306
5041,0.92,2006,33030.0,0.6,201402,0.015851,0.015875,0.015935,0.015715
5042,0.96,32000,91529.0,0.6,201301,0.053068,0.053432,0.053751,0.054001
5043,0.96,2012,91529.0,0.6,201301,0.047085,0.047947,0.048873,0.047071


# Salvar/abrir DF em do drive

In [2]:
save = False # @param {type:"boolean"}
import pandas as pd

In [11]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def save_csv_to_drive(DF, filename):
  DF.to_parquet('/content/drive/My Drive/' + filename + ".parquet.gzip", index= False)

In [ ]:
if save:
  save_csv_to_drive(normalizedDf, "fipe_normalized")

In [12]:
normalizedDf = pd.read_parquet('/content/drive/My Drive/fipe_normalized.parquet.gzip')

normalizedDf

,marca,anoModelo,codigoFipe,siglaCombustivel,mesDesejado,target,mes-1,mes-2,mes-3
0,0.56,32000,160130.0,0.6,200312,0.100326,0.100807,0.104721,0.092562
1,0.56,2002,160130.0,0.6,200312,0.082643,0.081502,0.085518,0.076716
2,0.00,1997,400017.0,0.6,201212,0.003709,0.003731,0.003748,0.003761
3,0.00,1996,400017.0,0.6,201212,0.003266,0.003292,0.003317,0.003338
4,0.00,1995,400017.0,0.6,201212,0.003049,0.003103,0.003148,0.003189
...,...,...,...,...,...,...,...,...,...
5040,0.92,2007,33030.0,0.6,201402,0.016369,0.016394,0.016549,0.016306
5041,0.92,2006,33030.0,0.6,201402,0.015851,0.015875,0.015935,0.015715
5042,0.96,32000,91529.0,0.6,201301,0.053068,0.053432,0.053751,0.054001
5043,0.96,2012,91529.0,0.6,201301,0.047085,0.047947,0.048873,0.047071


# Funções simples

In [157]:
def percent_error(x,y):
  return 100*(x-y)/y

In [158]:
def error_metrics(X, Y):
  PE = []
  for x, y in zip (X, Y):
    PE.append(percent_error(x,y))
  avg_percent_error = np.array([abs(e) for e in PE]).mean()
  shoot             = np.array([e for e in PE]).mean()
  return avg_percent_error, shoot

## Repete o mês anteriors

In [134]:
mes_ant = normalizedDf[['mes-1']].to_numpy()
mes_atual = normalizedDf[['target']].to_numpy()
error_percent, shoot = error_metrics(mes_ant, mes_atual)
print(error_percent,shoot)

1.6744470921654928 0.6426313556651326


## Ajuste de polinomios

In [136]:
valores_ref = normalizedDf[['mes-1','mes-2', 'mes-3']].to_numpy()
targets = normalizedDf[['target']].to_numpy()
valores_ref[0], targets[0]

(array([0.10080706, 0.10472118, 0.09256235]), array([0.10032588]))

### Linear  A(x)+B

In [137]:
vals = []
for ref, target in zip(valores_ref, targets):
  reta = np.polyfit([1,2,3], ref, 1)
  vals.append(reta[0]*0+reta[1])

error_percent, shoot = error_metrics(vals, targets)
print(error_percent,shoot)


2.1471762790482263 0.17118836029477588


### logaritmico A log(x)+B

In [144]:
vals = []
for ref, target in zip(valores_ref, targets):
  reta = np.polyfit(np.log([4,3,2]), ref, 1)
  vals.append(reta[0]*np.log(5)+reta[1])

error_percent, shoot = error_metrics(vals, targets)
print(error_percent,shoot)


1.8925528360491044 0.3661200134085961


### exponencial Ae**(xb) (tira o log dos 2 lados, usa log(y))

In [151]:
from math import exp
vals = []
for ref, target in zip(valores_ref, targets):
  reta = np.polyfit([2,1,0], np.log(ref), 1)
  vals.append(exp(reta[0]*3)*exp(reta[1]))

error_percent, shoot = error_metrics(vals, targets)
print(error_percent,shoot)

2.181806235869222 0.32789982594766004


# IA classica

In [66]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [54]:
x= normalizedDf.drop(["target"], axis=1).to_numpy()
y= normalizedDf[['target']].to_numpy()

In [34]:
x_train = x[:-100]
y_train = y[:-100]

x_test = x[-100:]
y_test = y[-100:]

## Regressão Linear

In [35]:
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

LinearRegression()

In [162]:
y_pred = regr.predict(x_test)

In [163]:
error_percent, shoot = error_metrics(y_pred, y_test)
print(error_percent, shoot)

2.2870484919965364 1.0648815632148483


In [40]:
# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.5f" % mean_squared_error(y_test, y_pred)/y_test)
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [[-5.92166035e-05  1.70217010e-08 -2.68180807e-10  4.49567011e-04
   9.07860890e-09  1.17643519e+00 -1.79128628e-01 -6.90300203e-03]]
Mean squared error: 0.00000
Coefficient of determination: 1.00


# Rede Neural

In [ ]:
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F

## Definições da Rede

## Fully connected
(4x8 and 784x600)

In [ ]:
class car_predictor_FC(nn.Module):
  def __init__(self):
    super(car_predictor_FC, self).__init__()

    self.fc1 = nn.Linear(in_features=8, out_features=600)
    self.drop = nn.Dropout2d(0.25)
    self.fc2 = nn.Linear(in_features=600, out_features=1200)
    self.fc3 = nn.Linear(in_features=1200, out_features=600)
    self.fc4 = nn.Linear(in_features=600, out_features=120)
    self.fc5 = nn.Linear(in_features=120, out_features=1)

  def forward(self, x):
    out = x.view(x.size(0), -1) #hack no começo para linearizar a imagem
    out = self.fc1(out)
    out = self.drop(out)
    out = F.relu(out)
    out = self.fc2(out)
    out = F.relu(out)
    out = self.fc3(out)
    out = self.drop(out)
    out = F.relu(out)
    out = self.fc4(out)
    out = F.relu(out)
    out = self.fc5(out)

    return out

## Data Loader

In [8]:
class CarDataLoader(Dataset):
    def __init__(self, DataFrame):
        super(CarDataLoader,self).__init__()
        self.target = DataFrame[['target']]
        self.info   = DataFrame.drop(["target"], axis=1)


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
      return torch.from_numpy(self.info.iloc[idx].to_numpy(dtype = np.float32)), torch.from_numpy(self.target.iloc[idx].to_numpy(dtype = np.float32))

NameError: name 'Dataset' is not defined

In [ ]:
testDS = CarDataLoader(normalizedDf)

train_loader = DataLoader(
    testDS, batch_size=4, shuffle=True, num_workers=0, pin_memory=True,
)

print (next(iter(train_loader)))

[tensor([[9.6000e-01, 1.9950e+03, 9.0107e+04, 6.0000e-01, 2.0130e+05, 1.4359e-02,
         1.4097e-02, 1.3765e-02],
        [8.9000e-01, 1.9990e+03, 2.3009e+05, 6.0000e-01, 2.0061e+05, 1.9641e-02,
         1.9705e-02, 1.9834e-02],
        [0.0000e+00, 1.9990e+03, 4.2030e+04, 6.0000e-01, 2.0011e+05, 7.1953e-03,
         7.2788e-03, 7.7288e-03],
        [9.6000e-01, 3.2000e+04, 9.0913e+04, 6.0000e-01, 2.0071e+05, 1.3358e-01,
         1.3546e-01, 1.3483e-01]]), tensor([[0.0146],
        [0.0194],
        [0.0071],
        [0.1357]])]


## Configurar GPU

In [ ]:
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

False


## Criar Modelo Base

In [ ]:
error = nn.CrossEntropyLoss()

learning_rate = 0.001

In [ ]:
BasicPredictor = car_predictor_FC()

print(BasicPredictor)
total_params = sum(p.numel() for p in BasicPredictor.parameters())
print(f"Number of parameters: {total_params}")

BasicPredictor.to(device)

BasicOptimizerCNN = torch.optim.Adam(BasicPredictor.parameters(), lr=learning_rate)


car_predictor_FC(
  (fc1): Linear(in_features=8, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=1200, bias=True)
  (fc3): Linear(in_features=1200, out_features=600, bias=True)
  (fc4): Linear(in_features=600, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=1, bias=True)
)
Number of parameters: 1519441


prepara os datasets para serem lidos

In [ ]:
def trainModel(model, optimizer, num_epochs):
  count = 0
  # Lists for visualization of loss and accuracy
  loss_list = []
  iteration_list = []
  accuracy_list = []

  # Lists for knowing classwise accuracy
  predictions_list = []
  labels_list = []
  for epoch in range(num_epochs):
      for info, labels in train_loader:
          # Transfering images and labels to GPU if available
          info, labels = info.to(device), labels.to(device)

          train = Variable(info)
          labels = Variable(labels)

          # Forward pass
          outputs = model(train)
          loss = error(outputs, labels)

          # Initializing a gradient as 0 so there is no mixing of gradient among the batches
          optimizer.zero_grad()

          #Propagating the error backward
          loss.backward()

          # Optimizing the parameters
          optimizer.step()

          count += 1

      # Testing the model
          if not (count % 50):    # It's same as "if count % 50 == 0"
              total = 0
              correct = 0

              for images, labels in test_loader:
                  info, labels = info.to(device), labels.to(device)
                  labels_list.append(labels)

                  test = Variable(info)

                  outputs = model(test)

                  predictions = torch.max(outputs, 1)[1].to(device)
                  predictions_list.append(predictions)
                  correct += (predictions == labels).sum()

                  total += len(labels)

              accuracy = correct * 100 / total
              loss_list.append(loss.data)
              iteration_list.append(count)
              accuracy_list.append(accuracy)

          if not (count % 500):
              print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

  return iteration_list, loss_list, accuracy_list, predictions_list, labels_list

In [ ]:
#train_set =
#test_set =

#batchSize = 100

#train_loader = torch.utils.data.DataLoader(train_set, batch_size=batchSize)
#test_loader  = torch.utils.data.DataLoader(test_set , batch_size=batchSize)

In [ ]:
test_loader = train_loader
iteration_list, loss_list, accuracy_list, predictions_list, labels_list = trainModel(BasicPredictor, BasicOptimizerCNN, 5)

Iteration: 500, Loss: -0.0, Accuracy: 0.0%
Iteration: 1000, Loss: -0.0, Accuracy: 0.0%
Iteration: 1500, Loss: -0.0, Accuracy: 0.0%
Iteration: 2000, Loss: -0.0, Accuracy: 0.0%
Iteration: 2500, Loss: -0.0, Accuracy: 0.0%


KeyboardInterrupt: 